In [78]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from logic.processing import load_data, preproc
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import SGDClassifier
from imblearn.pipeline import Pipeline as ImbPipeline

In [11]:
def data_filter(df,uselfCount=0,min_length=30):
    df = df.dropna()
    df = df[df['rating'].isin([1,2,3,8,9,10])]
    df['sentiment'] = 0
    df['sentiment'] = (df['rating'].isin([8,9,10])).astype(int)
    df = df[df['usefulCount']>uselfCount]
    df['review_length'] = df['review'].apply(lambda x: len(str(x).split()))
    df = df[df.review_length>=min_length]
    return df

In [79]:
def data_filter2(df,uselfCount=0,min_length=20):
    df = df.dropna()
    df['sentiment'] = df['rating'].apply(lambda x: 'Good' if x >= 4 else 'Bad')
    df = df[df['usefulCount']>uselfCount]
    df['review_length'] = df['review'].apply(lambda x: len(str(x).split()))
    df = df[df.review_length>=min_length]
    X = df[['review']] 
    y = df['sentiment'] 

    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X, y)
    

    df_resampled = pd.DataFrame(X_resampled, columns=['review'])
    df_resampled['sentiment'] = y_resampled
    
    return df_resampled
    return df

In [105]:
def balance_dataset(X, y):
    
    df = pd.DataFrame({'review': X, 'sentiment': y})

    good_reviews = df[df['sentiment'] == 'Good']
    bad_reviews = df[df['sentiment'] == 'Bad']
    

    min_size = min(len(good_reviews), len(bad_reviews))
    

    good_balanced = good_reviews.sample(n=min_size, random_state=42)
    bad_balanced = bad_reviews.sample(n=min_size, random_state=42)
    

    balanced_df = pd.concat([good_balanced, bad_balanced]).sample(frac=1, random_state=42)
    

    X_balanced = balanced_df['review']
    y_balanced = balanced_df['sentiment']
    
    return X_balanced, y_balanced

In [106]:
df_train = load_data('drugsComTrain_raw.csv')
df_test = load_data('drugsComTest_raw.csv')

In [107]:
df_train_filter = data_filter(df_train)
df_test_filter = data_filter(df_test)

In [108]:
df_train_prep = preproc(df_train_filter)
df_test_prep = preproc(df_test_filter)

In [96]:
X_train = df_train_prep['clean']
y_train = df_train_prep['sentiment']

In [97]:
X_test = df_test_prep['clean']
y_test = df_test_prep['sentiment']

In [109]:
X_train_balanced, y_train_balanced = balance_dataset(X_train,y_train)

In [110]:
y_train_balanced.value_counts()

sentiment
Good    30212
Bad     30212
Name: count, dtype: int64

In [29]:
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer(max_df=0.75, min_df=5, ngram_range=(1, 2))),
    ('undersample', RandomUnderSampler(random_state=42)),
    ('SGD', SGDClassifier(loss='hinge', penalty='l2', alpha=1/10, max_iter=100))
])

In [1]:
pipeline2 = make_pipeline(
    TfidfVectorizer(max_df=0.75, min_df=5, ngram_range=(1, 2)),
    SGDClassifier(loss='hinge', penalty='l2', alpha=1/1000, max_iter=1000, class_weight='balanced')
)

NameError: name 'make_pipeline' is not defined

In [113]:
pipeline2.fit(X_train_balanced,y_train_balanced)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, min_df=5, ngram_range=(1, 2))),
                ('sgdclassifier',
                 SGDClassifier(alpha=0.001, class_weight='balanced',
                               max_iter=100))])

In [114]:
y_pred = pipeline.predict(X_test)

In [115]:
y_pred_labels = ['Good' if pred == 1 else 'Bad' for pred in y_pred]

In [116]:
print(classification_report(y_test, y_pred_labels))

/Users/Flotchi/.pyenv/versions/3.10.6/envs/Mental_Health_Risks/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00     10267
        Good       0.50      1.00      0.67     10267

    accuracy                           0.50     20534
   macro avg       0.25      0.50      0.33     20534
weighted avg       0.25      0.50      0.33     20534



/Users/Flotchi/.pyenv/versions/3.10.6/envs/Mental_Health_Risks/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Flotchi/.pyenv/versions/3.10.6/envs/Mental_Health_Risks/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
